# SZYMON WYSOGLĄD

### Problem szeregowania zadań – algorytm Johnsona - CDS dla problemu 5 maszyn

In [59]:
import numpy as np
import pandas as pd

### Zadanie 1 - implementacja algorytmu CDS

In [60]:
class Matrix:
  def __init__(self, numberOfMachines, numberOfJobs):
    self.numberOfMachines = numberOfMachines
    self.numberOfJobs = numberOfJobs
    self.matrix = np.zeros((numberOfMachines, numberOfJobs))
    self.orders = set()

  def setMatrix(self, matrix):
    self.matrix = matrix
    
  def generateOrderPermutations(self):
    # O(n*m * 2m) = O(2nm^2), n to liczba zadań, m to liczba maszyn
    """
    Funkcja generuje możliwe kolejności na podstawie heurystyki CDS
    """
    compressedMatrix = np.zeros((2, self.numberOfJobs))
    n = self.numberOfMachines - 1
    for i in range(n):
      #start i end pomogą we wpisywaniu tasków w odpowiedniej kolejności
      start = 0 
      end = self.numberOfJobs - 1
      
      compressedMatrix[0, :] += self.matrix[i, :] #pierwszy wiersz dodajemy od przodu
      compressedMatrix[1, :] += self.matrix[n - i, :] # ostatni wiersz dodajemy od tyłu
      
      # macierz taken określa, czy dana kolumna została użyta
      taken = np.zeros((2, self.numberOfJobs))
      order = [0] * self.numberOfJobs
      for _ in range(self.numberOfJobs):
        minY, minX, _ = self.findMinimum(compressedMatrix, taken)
        taken[:, minX] = 1 # zadanie zostało wzięte
        if minY == 0: 
          # jeżeli zadanie jest w pierwszym wierszu, 
          # to dodajemy od początku
          order[start] = minX
          start += 1
        else:
          # w przeciwnym wypadku od końca
          order[end] = minX
          end -= 1
      self.orders.add(tuple(order))
    
    return self.orders      
      
  def findMinCost(self):
    # dla każdego order (max n, bo heurystyka) sprawdzamy, jaki jest koszt
    # złożoność O(n*n + n*m*n) = O(mn^2)
    minCost = np.inf
    bestOrder = None
    bestTimeMatrix = None
    for order in self.orders:
      orderedMatrix = self.constructMatrixByOrder(order) #O(n)
      timeMatrix = self.countTimes(orderedMatrix) #O(n*m)
      if timeMatrix[-1, -1] < minCost:
        minCost = timeMatrix[-1, -1]
        bestOrder = order
        bestTimeMatrix = timeMatrix
    
    return bestOrder, bestTimeMatrix, minCost
      
  def constructMatrixByOrder(self, order):
    # O(n), n to liczba zadań
      orderedMatrix = np.zeros((self.numberOfMachines, self.numberOfJobs))
      # ustawienie zadań macierzy pomocniczej w odpowiedniej kolejności
      for i, number in enumerate(order):
        orderedMatrix[:, i] = self.matrix[:,number]
      return orderedMatrix
    
  def countTimes(self, orderedMatrix):
    #O(n*m), n to liczba zadań, m to liczba maszyn
    """
    zwraca macierz sumaryczna kosztów dla macierzy zadań w określonej kolejności 
    """
    compressedMatrix = np.zeros((self.numberOfMachines, self.numberOfJobs))
    Y, X = orderedMatrix.shape
    for i in range(Y):
      for j in range(X):
        #jeżeli to pierwsze zadanie na pierwszej maszynie
        if j == 0 and i == 0:
          compressedMatrix[i, j] = orderedMatrix[i, j] 
        # jeżeli to jest pierwsze zadanie, to czynności na maszynach będą następowały po sobie
        elif j == 0:
          compressedMatrix[i, j] = compressedMatrix[i-1, j] + orderedMatrix[i, j]
        # jeżeli to jest pierwsza maszyna, 
        # to czynności będą następowały po skończeniu wcześniejszego zadania na tej maszynie
        elif i == 0:
          compressedMatrix[i,j] =  compressedMatrix[i, j-1] + orderedMatrix[i, j]
        # w przeciwnym razie bierzemy czas, po którym się skończy nasz przedmiot obrabiać na poprzedniej maszynie
        # oraz zakończy się zadanie na następnej maszynie => tzn. bierzemy maksimum 
        else:
          compressedMatrix[i,j] = max(compressedMatrix[i, j-1], compressedMatrix[i -1, j]) + orderedMatrix[i, j]
    return compressedMatrix
          

  def findMinimum(self, compressedMatrix, taken): 
    #O(2n), ponieważ macierz jest 2xN, n to liczba zadań 
    minimum = np.inf
    minX = None
    minY = None
    for i, row in enumerate(compressedMatrix):
      for j, elem in enumerate(row):
        if elem < minimum and taken[i, j] == 0:
          minX = j
          minY = i
          minimum = elem
    return minY, minX, minimum
      
  def CDS(self):
    # O(2nm^2) + O(mn^2)
    """
    zwraca kolejność zadań, macierz kosztu i minimalny koszt
    """
    self.generateOrderPermutations() # O(2nm^2)
    return self.findMinCost() # O(mn^2)
    
  def showData(self, taskOrder = None):
    """
    Funkcja używająca biblioteki pandas, do wyświetlenia danych
    """
    if not taskOrder:
      print("Początkowa macierz kosztów:")
      headers = [f'task{i}' for i in range(self.numberOfJobs)]
      machines = [f'M{i}' for i in range(self.numberOfMachines)]
      table = pd.DataFrame(self.matrix, columns = headers, index = machines)
      print(table, '\n')
    else:
      print("Zoptymalizowana macierz kosztów")
      m = self.constructMatrixByOrder(taskOrder)
      headers = [f'task{i}' for i in taskOrder]
      machines = [f'M{i}' for i in range(self.numberOfMachines)]
      table = pd.DataFrame(m, columns = headers, index = machines)
      print(table, '\n')
    
  def showTimeEndings(self, timeMatrix, order):
    """
    Funkcja wypisuje macierz czasów zakończeń zadań na poszczególnych maszynach
    """
    headers = [f'task{i}' for i in order]
    machines = [f'M{i}' for i in range(self.numberOfMachines)]
    table = pd.DataFrame(timeMatrix, columns = headers, index = machines)
    print("Macierz czasów zakończeń zadań na poszczególnych maszynach:")
    print(table,"\n")
    

### Zadanie 2 - przykładowe uszeregowanie zadań

In [61]:
#Ustawienie ziarna generatora, dla powtarzalności wyników
np.random.seed(42)

# liczba zadań i liczba maszyn
ntasks = 10
nmachines = 5

# Generowanie losowej macierzy
minVal = 1
maxVal = 10
data = np.random.randint(minVal, maxVal, size=(nmachines, ntasks))

# ustawienie danych w macierzy 
m = Matrix(nmachines, ntasks)
m.setMatrix(data)

# wyświetlenie danych wejściowych
m.showData()

Początkowa macierz kosztów:
    task0  task1  task2  task3  task4  task5  task6  task7  task8  task9
M0      7      4      8      5      7      3      7      8      5      4
M1      8      8      3      6      5      2      8      6      2      5
M2      1      6      9      1      3      7      4      9      3      5
M3      3      7      5      9      7      2      4      9      2      9
M4      5      2      4      7      8      3      1      4      2      8 



In [62]:
order, costMatrix, cost = m.CDS()
m.showData(order)
m.showTimeEndings(costMatrix, order)

Zoptymalizowana macierz kosztów
    task5  task9  task3  task4  task7  task2  task1  task0  task6  task8
M0    3.0    4.0    5.0    7.0    8.0    8.0    4.0    7.0    7.0    5.0
M1    2.0    5.0    6.0    5.0    6.0    3.0    8.0    8.0    8.0    2.0
M2    7.0    5.0    1.0    3.0    9.0    9.0    6.0    1.0    4.0    3.0
M3    2.0    9.0    9.0    7.0    9.0    5.0    7.0    3.0    4.0    2.0
M4    3.0    8.0    7.0    8.0    4.0    4.0    2.0    5.0    1.0    2.0 

Macierz czasów zakończeń zadań na poszczególnych maszynach:
    task5  task9  task3  task4  task7  task2  task1  task0  task6  task8
M0    3.0    7.0   12.0   19.0   27.0   35.0   39.0   46.0   53.0   58.0
M1    5.0   12.0   18.0   24.0   33.0   38.0   47.0   55.0   63.0   65.0
M2   12.0   17.0   19.0   27.0   42.0   51.0   57.0   58.0   67.0   70.0
M3   14.0   26.0   35.0   42.0   51.0   56.0   64.0   67.0   71.0   73.0
M4   17.0   34.0   42.0   50.0   55.0   60.0   66.0   72.0   73.0   75.0 



### Zadanie 3 - pytania i odpowiedzi
<ol style="padding: 0; margin: 0;"> 
<li  style="margin-bottom: 20px;"> Jaki typ problemu rozwiązujemy (klasyfikacja Grahama)? 
<b><br/>Problem szeregowania zadań na maszynach, w którym zadania wykonywane są przez maszyny w identycznej kolejności - flow shop.</b>
</li>



<li  style="margin-bottom: 20px;"> Jakie czasy uzyskamy przy alternatywnych sposobach uszeregowania (takie samo
min dla kilku zadań)? 
<b> <br/> Z racji, ze algorytm CDS jest algorytmem heurystycznym, to dla alternatywnych sposobów uszeregowania mozemy uzyskać inne wyniki.</b>
</li>

<li  style="margin-bottom: 20px;"> Jakie warunki są konieczne w realizacji algorytmu / co jeśli nie będzie spełniony? 
<b><br/> Warunkiem jest posiadanie wszystkich przedmiotów do obróbki w gotowości w tym samym czasie. W przeciwnym razie problem, którego rozwiązania poszukujemy nie zostanie rozwiązany przez CDS. </br>
</li>

<li  style="margin-bottom: 20px;"> Jaka jest złożoność obliczeniowa algorytmu? 
<b><br/> W celu przejścia wszystkich kombinacji mielibyśmy złożoność O(n!), jednak algorytm heurystyczny pozwala nam na zmniejszenie tej liczby. Dzięki temu mamy O(n) możliwości ułożenia zadań, przez co złożoność obliczeniowa algorytmu to O(2nm^2) + O(mn^2), czyli O(n^3). </b>
</li>
</ol>